### Sequence Types

__Built-in__
- Mutable
    - lists, bytearrays
- Immutable
    - strings, tuples, range, bytes

Additional Standard Types:
- `collections` package:
    - namedtuple, deque
- `array` module:
    - array
    
__Homogeneous vs Heterogeneous Sequences__

Homogeneous: each element is of the same type, eg. strings

Heterogeneous: each element may be of a different type, eg. lists

__Iterable Type vs Sequence Types__

Iterable: a container object which can list out its elements one by one


In [1]:
# Every sequence type is an iterable
l = [1, 2, 3]

for e in l: ...
    
l[0]

1

In [2]:
# But an iterable is not necessarily a sequence type
s = {1, 2, 3}

for e in s: ...
    
s[0] # Cannot access by index!

TypeError: 'set' object is not subscriptable

__Standard Sequence Methods__


In [8]:
s = [1, 2, 3]
x = 1

# Buit-in sequences, both mutable and immutable, support the following methods:
x in s     # membership
x not in s
s += [1, 1, 1] # concatenation
s * 2      # repetition
len(s)
min(s)     # (if ordering b/w elements is defined (<, >, etc))
max(s)     #
s.index(x) # first occurence of x 
s.index(x, 3) # find x at or after index 3
s.index(x, 3, 5) # find x between index 3 and 5
s[0]       # index
s[0:3]     # elements from index 0(inclusive) to 3(exclusive)
s[0:5:2]   # same as above, but in steps of 2 (skip every other element)

[1, 3, 1]

NOTE: Slices will always return the same container type as the object being sliced

`range` objects are more restrictive:
- no concat/repetition
- min, max, `in`, `not in` are not as efficient

__Beware of Concatenation/Repetition of Mutable Objects__

In [9]:
# x is a list of immutable integers
x = [1, 2]
a = x + x

a[0] is a[1]

False

In [12]:
# x is a list of mutable lists (a 2d array)
x = [ [0, 0] ]
a = x + x

a[0] is a[1] # ! both elements in `a` are the same object in memory!

True

In [13]:
x = [ [0, 0] ]
a = x * 2

a[0] is a[1]

True

### Mutable Sequence Types

__Mutating Objects__

In [17]:
# Concatenation will create a new object in memory
names = ['Alice', 'Bob']
print(id(names))

names = names + ['Carol']
print(id(names))

140711249606464
140710753227456


In [18]:
# Using append mutates the original object in memory
names.append('Darren')
print(id(names))

140710753227456


__Mutating using `[]`__

In [19]:
x = 5
i = 0
j = 4
s = [1, 2, 3]
s2 = [9, 8, 7]

s[i] = x
s[i:j] = s2
del s[i]
del s[i:j]

__Methods of Mutable Sequence Types__

In [21]:
s.clear()
s.append(x)
s.insert(i, x)
s.extend([4, 5, 6])
s.pop(i)
s.remove(x)
s.reverse()
s.copy()

[6, 5, 4]

### Lists vs Tuples

__*Constant Folding*__ is the process of recognizing and evaluatng constant expressions at compile time rather than computing them at runtime.

In [22]:
from dis import dis

In [29]:
dis(compile('[1, 2, 3, "a"]', 'string', 'eval'))

  1           0 BUILD_LIST               0
              2 LOAD_CONST               0 ((1, 2, 3, 'a'))
              4 LIST_EXTEND              1
              6 RETURN_VALUE


In [30]:
dis(compile('(1, 2, 3, "a")', 'string', 'eval'))

  1           0 LOAD_CONST               0 ((1, 2, 3, 'a'))
              2 RETURN_VALUE


In [31]:
l1 = [1, 2, 3, 4, 5, 6, 7, 8, 9]
t1 = (1, 2, 3, 4, 5, 6, 7, 8, 9)

In [32]:
# Copying a list will result in a new object in memory
l2 = list(l1)

l1 is l2

False

In [33]:
# Where as copying a tuple will not, since it is immutable
t2 = tuple(t1)

t1 is t2

True

In [34]:
# Constructing lists takes significantly longer than creating tuples
from timeit import timeit

timeit('tuple((1, 2, 3, 4, 5))', number=5_000_000)

0.21645070799422683

In [35]:
timeit('list((1, 2, 3, 4, 5))', number=5_000_000)

0.4638821799962898

### Copying Sequences

We can copy a sequence using a variety of methods:

In [36]:
s = [10, 20, 30]

# Simple Loop (not very pythonic!)
cp = []
for e in s:
    cp.append(e)

# List Comprehension
cp = [e for e in s]

# Using copy()
cp = s.copy()

# Slicing
cp = s[:]

# Using list()
cp = list(s)

__Shallow Copies__

The above methods will create a new *list* object, but the references inside the list will be the same in both the original and the copy

In [42]:
s = [ [10, 20], [30, 40] ]
cp = s.copy()

# Here we can modify the copy list itself and not affect the original
cp[0] = 'python'
print(f's: {s}\n cp: {cp}\n')

# But if we modify a mutable object inside the copy, we affect the original list
cp[1][0] = 100
print(f's: {s}\n cp: {cp}\n')

s: [[10, 20], [30, 40]]
 cp: ['python', [30, 40]]

s: [[10, 20], [100, 40]]
 cp: ['python', [100, 40]]



__Deep Copies__

If collections contain mutable elements, shallow copies ae not sufficient to ensure the copy can never be able to modify the original. Instead we can use *deep copies*.

In general, objects know how to make *shallow* copies of themselves, eg. lists, sets, dictionaries all have a `copy()` method.

There is also the `copy` module from the standard library, which has `copy` and `deepcopy` operations.

In [44]:
class MyClass:
    def __init__(self, a):
        self.a = a

In [45]:
from copy import copy, deepcopy

In [46]:
x = [10, 20]
obj = MyClass(x)

x is obj.a

True

In [47]:
shallow_cp = copy(obj)

shallow_cp.a is obj.a

True

In [48]:
deep_cp = deepcopy(obj)

deep_cp.a is obj.a

False

### Slicing

Slicing relies on indexing, so it only works with *sequence* types.

__The `slice` Type__

Although we usually slice seqs using the conventional syntax: `my_list[i:j]`, slice definitions (`[i:j]`) are actually objects of type `slice`.

In [52]:
# This is useful if we want to name slices and use symbols instead of literals
s = slice(0, 2)
type(s)

slice

In [53]:
l = [1, 2, 3, 4]

l[s]

[1, 2]

__Effective Start and Stop Bounds__

In [55]:
# Interestingly the following works:
l = [1, 2, 3, 4, 5]

l[3:100] # We can specify slices that are 'out of bounds' of the sequence they are applied to

[4, 5]

In [57]:
# Negative indices also work:
l[-1]

l[-3:-1]

[3, 4]

__Step Value__

In [59]:
# Slicing supports a third argument, the step or stride value
l[0:6:2]

[1, 3, 5]

In [60]:
# You can give a negative value for the stride as well, which will iterate backwards
l[::-1]

[5, 4, 3, 2, 1]

__Range Equivalence__

Any slice essentially defines a sequence of indices that is used to select elements for another sequence. Any indices defined by a slice can also be defined using a `range`. The difference is that slices are defined independently of the sequence being sliced. And the equivalent range is only calculated once the length of the sequence being sliced is known.

eg.
`[0:100]` applied to -> `[1, 2, 3, 4, 5]` equals -> `range(0, 5)`


| Case         | `[i:j:k] k > 0`        | `[i:j:k] k < 0`         |
| ------------ | ---------------------- | ----------------------- |
| i > len(seq) | i = len(seq)           | i = len(seq)-1          |
| j > len(seq) | j = len(seq)           | j = len(seq)-1          |
| i < 0        | i = max(0, len(seq)+i) | i = max(-1, len(seq)+i) |
| j < 0        | j = max(0, len(seq)+j) | j = max(-1, len(seq)+j) |
| i is None    | i = 0                  | i = len(seq)-1          |
| j is None    | j = len(seq)           | j = -1                  |

The `slice` object has a method, `indices`, that returns the equivalent range start/stop/step for any slice given the length of the sequence being sliced. The values in this tuple can be used to generate a list of indices using the range function.

In [65]:
slc = slice(10, -5, -1)

rng = slc.indices(6) # 6 is length of the sequence we want to slice

print(list(range(*rng)))

[5, 4, 3, 2]


### Custom Sequences

At its most basic, an immutable sequence type should support:
- returning the length of the sequence (`__len__`)
- returning the element at a given index (`__getitem__`)

__The `_ _getitem_ _` Method__

The getitem method should return an element of the sequence based on the given index, or raise an `IndexError` if the index is out of bounds. The method may also support negative indices and slicing.

In [1]:
a_list = [1, 2, 3]

a_list.__getitem__(0)
a_list.__getitem__(-1)
a_list.__getitem__(slice(None, None, -1))

[3, 2, 1]

__The `_ _len_ _` Method__

In [2]:
a_list.__len__()

3

### Concatenation and Repetition

__Concatenation using `+`__

Using the `+` operator will result in a new object in memory.

In [5]:
l1 = [1, 2, 3]
l2 = [4, 5, 6]
print(id(l1))

l1 = l1 + l2
print(id(l1))

140125596886272
140125596965440


__In-Place Concatenation using `+=`__

While using `+=` for numbers always results in a new object, this is not the case for everything. For lists, `+=` will extend the list, modifying the original list object.

In [6]:
l1 = [1, 2, 3]
l2 = [4, 5, 6]
print(id(l1))

l1 += l2
print(id(l1))

140125865618304
140125865618304


__In-Place Repetition using `*=`__

The `*=` operator functions similarly to `+=` for lists. 

In [7]:
l = [1, 2, 3]
print(id(l))

l *= 2
print(id(l))

140125596957632
140125596957632


### Assignments in Mutable Sequences

__Assigning Values via Index, Slices, and Extended Slices__

Mutable sequences support assignments via a specific index, but also support assignment via slices. The value being assigned via slicing *must* be an iterable.

In [13]:
l = [1, 2, 3, 4, 5]

# For non-extended slices, the length does not need to be the same
l[1:3] = (10, 20, 30)
l

[1, 10, 20, 30, 4, 5]

In [14]:
l = [1, 2, 3, 4, 5]

# for extended slices, the length must be the same
l[0:4:2] = [10, 30]
l

[10, 2, 30, 4, 5]

In [15]:
l = [1, 2, 3, 4, 5]

# Assigning an empty iterable essentially deletes the elements of the slice
l[2:3] = []
l

[1, 2, 4, 5]

In [17]:
l = [1, 2, 3, 4, 5]

# Inserting via slices
l[1:1] = 'abc'
l

[1, 'a', 'b', 'c', 2, 3, 4, 5]

### Custom Sequences

__Overloading Concatenation and Repetition__

We can overload the definiton of the `+`, `+=`, `*`, and `*=` operators for our custom sequences by writing the following dunder methods, respectively:
- `__add__`
- `__iadd__`
- `__mul__`
- `__imul__`

In general, we expect:
- `obj1 + obj2`: obj1 and obj2 are of the same type, and the result is a new object also of the same type
- `obj1 += obj2`: obj2 is any iterable, and the result is the original memory reference of obj1
- `obj * n`: n is non-negative and the result is a new object of the same type as obj1
- `obj *= n`: n is non-negative and the result is the original obj1 reference

__Overloading Assignment__

Similarly to accessing elements of a custom sequence, we can implement the `__setitem__` method to overload assignments.

__Additional Functions and Operators__

- `__contains__`: `in`
- `__delitem__`: `del`
- `__rmul__`: `n * seq`
    - If Python cannot multiply the left operand to the right, it will attempt to multiply the left operand with the right, which calls `__rmul__`

### Sorting

Python provides the `sorted()` function which will sort a given iterable. The default sort direction is *ascending*, but this can be changed by passing `reverse=True` to the sorted function. 

The `sorted` function:
- makes a copy of the iterable
- returns the sorted elements in a list
- uses the TimSort algorithm, a stable sort

What to do when items are not pair-wise comparable?

__Sort Keys__

The `sorted` function accepts a keyword-only argument, `key`. The key arguemnt must be a function that returns the __sort key__ for any given element in the sequence being sorted.

Sort keys need not be numerical, they just need to have a natural sort order (`<` or `>`).

For the 'natural' sort of elements, we can always think of the keys as the elements themselves.

In [26]:
class Person:
    def __init__(self, age):
        self.age = age
    
    def __repr__(self):
        return f"Person of {self.age} years of age"

In [27]:
p1 = Person(20)
p2 = Person(30)
p3 = Person(40)

In [28]:
sorted((p1, p2, p3), key=lambda p: p.age)

[Person of 20 years of age,
 Person of 30 years of age,
 Person of 40 years of age]

__In-Place Sorting__

If the iterable is mutable, in-place sorting is *possible*. It depends on the particular type  you are dealing with. Pythons `list` object supports in-place sorting through its `sort` method.

In [30]:
l = [4, 2, 7, 5, 0]
l.sort()
l

[0, 2, 4, 5, 7]

### List Comprehensions

The goal of list comprehensions is to generate a new list by transforming, and optionally filtering, another iterable.

In [1]:
# You can split a comprehension over multiple lines
sq = [i**2 
      for i in range(1, 101) 
      if i%2 and i&3 and i%5]

__Comprehension Internals__

Comprehensions have their own local scope, like a function. They can also access global and nonlocal variables, also like functions.

When the RHS of a comprehension statement is *compiled*, Python creates a temporary function that will be used to evaliate the comprehension. When the statement is *executed*, Python executes the temporary function, stores the new list in memory and points the variable to that object.

__Nested Comprehensions__

In [2]:
# A nested comprehension can access (nonlocal) variables from the
# enclosing comprehension
res = [ [i * j for j in range(5)] for i in range(5) ]

__Nested Loops in Comprehensions__

In [3]:
l = []
for i in range(5):
    for j in range(5):
        for k in range(5):
            l.append((i, j, k))

# We can rewrite this as a comprehension with nested loops.
# Note the order of the nesting remains the same as above.
l = [(i, j, k) for i in range(5) for j in range(5) for k in range(5)]

In [4]:
l = []
for i in range(1, 6):
    if i%2 == 0:
        for j in range(1, 6):
            if j%3 == 0:
                l.append((i, j))
                
[(i, j) 
 for i in range(1, 6) if i%2==0 
 for j in range(1, 6) if j%3==0]

[(2, 3), (4, 3)]